In [1]:
import os

import numpy as np
import pandas as pd

# 2020 Summer

# Control

In [2]:
DIRECTORY = '../ch2-1_weighing/data/2020_S/'
file_list = os.listdir(DIRECTORY)
dataset_list = [file for file in file_list if file.endswith('.dat')]
dataset_list.sort()

In [3]:
dataset_list

['CR1000_Table1_2020_05_06_21_19_15.dat',
 'CR1000_Table1_2020_05_13_21_19_15.dat',
 'CR1000_Table1_2020_05_25_14_35_46.dat',
 'CR1000_Table1_2020_06_10_16_29_39.dat',
 'CR1000_Table1_2020_06_18_15_51_17.dat',
 'CR1000_Table1_2020_07_13_15_19_58.dat']

In [ ]:
greenhouse_df = []
for FILENAME in dataset_list:
    temp_df = pd.read_csv(DIRECTORY + FILENAME, sep=',', index_col='TIMESTAMP', skiprows=[0, 2, 3])
    greenhouse_df.append(temp_df)
greenhouse_df = pd.concat(greenhouse_df)

In [ ]:
greenhouse_df.index = pd.DatetimeIndex(greenhouse_df.index)

In [ ]:
cr1000_df = greenhouse_df
cr1000_df = cr1000_df[['Loadcell_1', 'Loadcell_2', 'Pyrano_Wsec_1', 'Temp_Avg', 'Humid_Avg']]

In [ ]:
date_index = pd.date_range(cr1000_df.index[0], cr1000_df.index[-1], freq='1 min')

In [ ]:
cr1000_df = cr1000_df.reindex(date_index)

In [ ]:
DIRECTORY = './data/2020_S/'
file_list = os.listdir(DIRECTORY)
dataset_list = [file for file in file_list if file.endswith('.xlsx')]
dataset_list.sort()

In [ ]:
greenhouse_df = []
divider = 0
for FILENAME in dataset_list:
    if 'A2' in FILENAME:
        divider += 1
    temp_df = pd.read_excel(DIRECTORY + FILENAME, index_col='date')
    temp_df.index = pd.DatetimeIndex(temp_df.index)
    temp_df = temp_df[['weight', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permittivity']]
    greenhouse_df.append(temp_df)

In [ ]:
for i in range(len(greenhouse_df)):
    if greenhouse_df[i].index.round('1 min')[0].minute == 1:
        greenhouse_df[i].index = (greenhouse_df[i].index.round('1 min') - pd.Timedelta('1 min'))
    elif greenhouse_df[i].index.round('1 min')[0].minute == 2:
        greenhouse_df[i].index = (greenhouse_df[i].index.round('2 min') - pd.Timedelta('2 min'))
    else:
        greenhouse_df[i].index = greenhouse_df[i].index.round('1 min')

In [ ]:
for i in range(len(greenhouse_df)):
    greenhouse_df[i] = greenhouse_df[i].groupby(greenhouse_df[i].index).first()

In [ ]:
date_index = pd.date_range(greenhouse_df[0].index[0], greenhouse_df[-1].index[-1], freq='1 min')

In [ ]:
_1 = pd.concat(greenhouse_df[:divider]).reindex(date_index)
_2 = pd.concat(greenhouse_df[divider:]).reindex(date_index)

In [ ]:
_1.columns = ['Loadcell_3', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permittivity']
_2.columns = ['Loadcell_4', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permittivity']

In [ ]:
_1

In [ ]:
SW2_df = pd.concat([_1.loc[pd.date_range(cr1000_df.index[0], _1.index[-1], freq='1 min')],
                    cr1000_df.loc[pd.date_range(cr1000_df.index[0], _1.index[-1], freq='1 min')]], axis=1)

In [ ]:
SW2_df.columns = ['loadcell_3', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permit', 'loadcell_1', 'loadcell_2', 'rad', 'temp', 'hum']

In [ ]:
SW2_df = SW2_df[['temp', 'hum', 'rad', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permit', 'loadcell_1', 'loadcell_2', 'loadcell_3']]

In [ ]:
SW2_df['temp'] = ((SW2_df['temp']-1000)/80)
SW2_df['hum'] = ((SW2_df['hum']-1000)/40)
SW2_df['loadcell_1'] = ((SW2_df['loadcell_1']/100))
SW2_df['loadcell_2'] = ((SW2_df['loadcell_2']/100))

In [ ]:
SW2_df.to_csv('./results/2020_S/SW2_greenhouse_origin.csv')

# Far-red

In [ ]:
cr1000_a3_df = pd.read_csv('./data/2020_S/A3/A3_data.csv')
cr1000_a3_df['date'] = cr1000_a3_df['date'].str.cat(cr1000_a3_df['time'], sep=' ')
cr1000_a3_df.index = cr1000_a3_df['date']
cr1000_a3_df = cr1000_a3_df.drop(['date', 'time'], axis=1)
cr1000_a3_df.index = pd.DatetimeIndex(cr1000_a3_df.index)

In [ ]:
cr1000_a3_df = cr1000_a3_df.groupby(cr1000_a3_df.index).mean()
cr1000_a3_df = cr1000_a3_df.reindex(pd.date_range(cr1000_a3_df.index[0], cr1000_a3_df.index[-1], freq='1 min')).interpolate()
cr1000_a3_df['Pyrano_Wsec'] = (cr1000_a3_df.loc[:, 'Pyrano_Wsec_1'] + cr1000_a3_df.loc[:, 'Pyrano_Wsec_2'])/2
cr1000_a3_df = cr1000_a3_df[['Pyrano_Wsec', 'Temp_Avg', 'Humid_Avg']]

In [ ]:
SW2_df = pd.concat([_2.loc[pd.date_range(cr1000_a3_df.index[0], _2.index[-1], freq='1 min')],
                    cr1000_a3_df.loc[pd.date_range(cr1000_a3_df.index[0], _2.index[-1], freq='1 min')]], axis=1)

In [ ]:
SW2_df

In [ ]:
SW2_df.columns = ['loadcell_4', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permit', 'rad', 'temp', 'hum']

In [ ]:
SW2_df = SW2_df[['temp', 'hum', 'rad', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permit', 'loadcell_4']]

In [ ]:
SW2_df.to_csv('./results/2020_S/SW2_FR_greenhouse_origin.csv')

# 2020 Winter

# Control

In [ ]:
DIRECTORY = './data/2020_W/'
file_list = os.listdir(DIRECTORY)
dataset_list = [file for file in file_list if file.endswith('.dat')]
dataset_list.sort()

In [ ]:
dataset_list

In [ ]:
greenhouse_df = []
for FILENAME in dataset_list:
    try:
        temp_df = pd.read_csv(DIRECTORY + FILENAME, sep='\t', index_col='TIMESTAMP', skiprows=[0, 2, 3])
    except ValueError:
        temp_df = pd.read_csv(DIRECTORY + FILENAME, sep=',', index_col='TIMESTAMP', skiprows=[0, 2, 3])
    greenhouse_df.append(temp_df)
greenhouse_df = pd.concat(greenhouse_df, sort=True)

In [ ]:
greenhouse_df.index = pd.DatetimeIndex(greenhouse_df.index)

In [ ]:
cr1000_df = greenhouse_df
cr1000_df = cr1000_df[['Loadcell_1', 'Loadcell_2', 'Pyrano_Wsec_1', 'Temp_Avg', 'Humid_Avg']]

### 2020 Winter A3

In [ ]:
DIRECTORY = './data/2020_W/A3_CR1000/'
file_list = os.listdir(DIRECTORY)
dataset_list = [file for file in file_list if file.endswith('.dat')]
dataset_list.sort()

In [ ]:
dataset_list

In [ ]:
greenhouse_df = []
for FILENAME in dataset_list:
    try:
        temp_df = pd.read_csv(DIRECTORY + FILENAME, sep='\t', index_col='TIMESTAMP', skiprows=[0, 2, 3])
    except ValueError:
        temp_df = pd.read_csv(DIRECTORY + FILENAME, sep=',', index_col='TIMESTAMP', skiprows=[0, 2, 3])
    greenhouse_df.append(temp_df)
greenhouse_df = pd.concat(greenhouse_df, sort=True)

In [ ]:
greenhouse_df.index = pd.DatetimeIndex(greenhouse_df.index)

In [ ]:
greenhouse_df['Pyrano_Wsec'] = (greenhouse_df.loc[:, 'Pyrano_Wsec_1'] + greenhouse_df.loc[:, 'Pyrano_Wsec_2'])/2

In [ ]:
cr1000_a3_df = greenhouse_df
cr1000_a3_df = cr1000_a3_df[['Pyrano_Wsec', 'Temp_Avg', 'Humid_Avg']]

#### date range

In [ ]:
cr1000_df = cr1000_df.loc['2020-08-26 00:00:00':'2021-01-24 23:59:00']
cr1000_a3_df = cr1000_a3_df.loc['2020-08-26 00:00:00':'2021-01-24 23:59:00']

In [ ]:
date_index = pd.date_range(cr1000_df.index[0], cr1000_df.index[-1], freq='1 min')

In [ ]:
cr1000_a3_df = cr1000_a3_df.reindex(date_index)

In [ ]:
DIRECTORY = './data/2020_W/'
file_list = os.listdir(DIRECTORY)
dataset_list = [file for file in file_list if file.endswith('.xlsx')]
dataset_list.sort()

In [ ]:
dataset_list

In [ ]:
greenhouse_df = []
div_1 = 0
div_2 = 0
div_3 = 0
for FILENAME in dataset_list:
    if 'CT' in FILENAME:
        div_1 += 1
    if 'FR' in FILENAME:
        div_2 += 1
    if 'RB1' in FILENAME:
        continue
    if 'RB2' in FILENAME:
        div_3 += 1
    temp_df = pd.read_excel(DIRECTORY + FILENAME, index_col='date')
    temp_df.index = pd.DatetimeIndex(temp_df.index)
    temp_df = temp_df[['weight', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permittivity']]
    greenhouse_df.append(temp_df)

In [ ]:
for i in range(len(greenhouse_df)):
    if greenhouse_df[i].index.round('1 min')[0].minute == 1:
        greenhouse_df[i].index = (greenhouse_df[i].index.round('1 min') - pd.Timedelta('1 min'))
    elif greenhouse_df[i].index.round('1 min')[0].minute == 2:
        greenhouse_df[i].index = (greenhouse_df[i].index.round('2 min') - pd.Timedelta('2 min'))
    else:
        greenhouse_df[i].index = greenhouse_df[i].index.round('1 min')

In [ ]:
for i in range(len(greenhouse_df)):
    greenhouse_df[i] = greenhouse_df[i].groupby(greenhouse_df[i].index).first()

In [ ]:
date_index = pd.date_range(greenhouse_df[0].index[0], greenhouse_df[-1].index[-1], freq='1 min')

In [ ]:
other_df = []
for FILENAME in dataset_list:
    if 'RB1' in FILENAME:
        temp_df = pd.read_excel(DIRECTORY + FILENAME, index_col='date')
        temp_df.index = pd.DatetimeIndex(temp_df.index)
        temp_df = temp_df[['weight']]
        other_df.append(temp_df)
    else:
        continue

In [ ]:
ct_df = pd.concat(greenhouse_df[:div_1]).reindex(date_index)
fr_df = pd.concat(greenhouse_df[div_1:div_1+div_2]).reindex(date_index)
rb_df = pd.concat(greenhouse_df[div_1+div_2:]).reindex(date_index)

In [ ]:
ct_df.columns = ['Loadcell_3', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permittivity']

In [ ]:
SW2_ct_df = pd.concat([ct_df.loc[pd.date_range(cr1000_df.index[0], ct_df.index[-1], freq='1 min')],
                       cr1000_df.loc[pd.date_range(cr1000_df.index[0], ct_df.index[-1], freq='1 min')]], axis=1)

In [ ]:
SW2_ct_df = SW2_ct_df.astype('float')

In [ ]:
SW2_ct_df.columns = ['loadcell_3', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permit', 'loadcell_1', 'loadcell_2', 'rad', 'temp', 'hum']

In [ ]:
SW2_ct_df = SW2_ct_df[['temp', 'hum', 'rad', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permit', 'loadcell_1', 'loadcell_2', 'loadcell_3']]

In [ ]:
SW2_ct_df['temp'] = ((SW2_ct_df['temp']-1000)/80)
SW2_ct_df['hum'] = ((SW2_ct_df['hum']-1000)/40)
SW2_ct_df['loadcell_1'] = ((SW2_ct_df['loadcell_1']/100))
SW2_ct_df['loadcell_2'] = ((SW2_ct_df['loadcell_2']/100))

In [ ]:
SW2_ct_df.loc[:, 'loadcell_1'] = SW2_ct_df.loc[:, 'loadcell_1'] +11/100
SW2_ct_df.loc[:, 'loadcell_2'] = SW2_ct_df.loc[:, 'loadcell_2'] -388/100 #calibration
SW2_ct_df.loc[:, 'loadcell_3'] = SW2_ct_df.loc[:, 'loadcell_3']*2 -8.3

In [ ]:
SW2_ct_df.to_csv('./results/2020_W/SW_CT_greenhouse_origin.csv')

In [ ]:
SW2_ct_df.loc[:, 'loadcell_1'] = SW2_ct_df.loc[:, 'loadcell_1'] - 0.8
SW2_ct_df.loc[:, 'loadcell_2'] = SW2_ct_df.loc[:, 'loadcell_2'] - 0.5
SW2_ct_df.loc[:, 'loadcell_3'] = SW2_ct_df.loc[:, 'loadcell_3'] - 0.8

In [ ]:
other_df = pd.concat(other_df).reindex(date_index)

In [ ]:
rb_df['weight2'] = other_df.values

In [ ]:
fr_df.columns = ['Loadcell_4', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permittivity']
rb_df.columns = ['Loadcell_6', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permittivity', 'Loadcell_5']

In [ ]:
SW2_fr_df = pd.concat([fr_df.loc[pd.date_range(cr1000_a3_df.index[0], fr_df.index[-1], freq='1 min')],
                       cr1000_a3_df.loc[pd.date_range(cr1000_a3_df.index[0], fr_df.index[-1], freq='1 min')]], axis=1)
SW2_rb_df = pd.concat([rb_df.loc[pd.date_range(cr1000_a3_df.index[0], rb_df.index[-1], freq='1 min')],
                       cr1000_a3_df.loc[pd.date_range(cr1000_a3_df.index[0], rb_df.index[-1], freq='1 min')]], axis=1)

In [ ]:
SW2_fr_df.columns = ['loadcell_4', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permit', 'rad', 'temp', 'hum']
SW2_rb_df.columns = ['loadcell_6', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permit', 'loadcell_5', 'rad', 'temp', 'hum']

In [ ]:
SW2_fr_df = SW2_fr_df[['temp', 'hum', 'rad', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permit', 'loadcell_4']]
SW2_rb_df = SW2_rb_df[['temp', 'hum', 'rad', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permit', 'loadcell_5', 'loadcell_6']]

In [ ]:
SW2_fr_df['temp'] = ((SW2_fr_df['temp']-1000)/80)
SW2_fr_df['hum'] = ((SW2_fr_df['hum']-1000)/40)
SW2_fr_df['loadcell_4'] = ((SW2_fr_df['loadcell_4']))

SW2_rb_df['temp'] = ((SW2_rb_df['temp']-1000)/80)
SW2_rb_df['hum'] = ((SW2_rb_df['hum']-1000)/40)
SW2_rb_df['loadcell_5'] = ((SW2_rb_df['loadcell_5']))
SW2_rb_df['loadcell_6'] = ((SW2_rb_df['loadcell_6']))

In [ ]:
SW2_fr_df.to_csv('./results/2020_W/SW_FR_greenhouse_origin.csv')
SW2_rb_df.to_csv('./results/2020_W/SW_RB_greenhouse_origin.csv')